In [1]:
!pip install yfinance
!pip install tqdm 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm


# -----------------------
# Helpers
# -----------------------

def normalize_ticker(raw):
    raw = raw.strip()

    if ":" in raw:
        exch, code = raw.split(":", 1)
        exch = exch.upper()

        if exch in ("TYO", "TSE", "JPX"):
            return f"{code}.T"
        if exch == "NASDAQ":
            return code
        if exch == "NYSE":
            return code

    return raw


def read_studios(path="studios.txt"):
    studios = []

    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()

            if not line or line.startswith("#"):
                continue

            if "," not in line:
                continue

            name, tick = line.split(",", 1)
            studios.append((name.strip(), normalize_ticker(tick)))

    print(f"Loaded {len(studios)} studios")
    return studios


def build_quarter_index(series):
    start = series.index.min()
    end = series.index.max()
    return pd.period_range(start=start, end=end, freq="Q")


# -----------------------
# Main
# -----------------------

def main():
    studios = read_studios("studios.txt")

    company_data = {}
    all_quarters = set()

    for name, ticker in tqdm(studios, desc="Fetching data"):
        try:
            tk = yf.Ticker(ticker)
            info = tk.info or {}
            shares = info.get("sharesOutstanding")

            hist = tk.history(period="max", auto_adjust=False)

            if hist.empty:
                print(f"NO DATA: {name} ({ticker})")
                continue

            if shares and shares > 0:
                hist["MC"] = hist["Close"] * shares
            else:
                hist["MC"] = hist["Close"]

            hist["Quarter"] = hist.index.to_period("Q")
            q = hist.groupby("Quarter")["MC"].last()

            if q.notna().sum() < 2:
                print(f"INSUFFICIENT DATA: {name}")
                continue

            company_data[name] = q
            all_quarters.update(q.index)

        except Exception as e:
            print(f"FAILED {name}: {e}")

    # -----------------------
    # Build master table
    # -----------------------

    quarters = sorted(all_quarters)
    master = pd.DataFrame(index=quarters)

    for name, series in company_data.items():
        full_q = build_quarter_index(series)
        series = series.reindex(full_q)
        series = series.interpolate(method="linear")
        master[name] = series.reindex(quarters)

    master["TotalValue"] = master.sum(axis=1)

    # -----------------------
    # CSV
    # -----------------------

    master.reset_index().to_csv(
        "quarterly_linear_interpolated.csv",
        index=False,
        float_format="%.2f"
    )

    # -----------------------
    # XML
    # -----------------------

    root = ET.Element("QuarterlyValues")

    for q in master.index:
        q_el = ET.SubElement(root, "Quarter", value=str(q))
        total = ET.SubElement(q_el, "TotalValue")
        total.text = f"{master.at[q, 'TotalValue']:.2f}"

        for name in company_data:
            val = master.at[q, name]
            if pd.notna(val):
                c = ET.SubElement(q_el, "Company", name=name)
                c.text = f"{val:.2f}"

    ET.ElementTree(root).write(
        "quarterly_linear_interpolated.xml",
        encoding="utf-8",
        xml_declaration=True
    )

    print("\nDONE")
    print("Files created:")
    print("- quarterly_linear_interpolated.csv")
    print("- quarterly_linear_interpolated.xml")


if __name__ == "__main__":
    main()


Loaded 39 studios


Fetching data:   0%|          | 0/39 [00:00<?, ?it/s]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:   3%|▎         | 1/39 [00:02<01:20,  2.11s/it]HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WIT STUDIO, SIGNAL.MD), TYO:3791"}}}
$WIT STUDIO, SIGNAL.MD), TYO:3791: possibly delisted; no timezone found
Fetching data:   5%|▌         | 2/39 [00:03<01:07,  1.82s/it]

NO DATA: IG Port (Production I.G (Wit Studio, Signal.MD), TYO:3791)


C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:   8%|▊         | 3/39 [00:04<00:51,  1.44s/it]HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: CRUNCHYROLL), TYO:6758"}}}
$CRUNCHYROLL), TYO:6758: possibly delisted; no timezone found
Fetching data:  10%|█         | 4/39 [00:06<00:49,  1.41s/it]

NO DATA: Sony Group (Aniplex (Crunchyroll), TYO:6758)


...

NO DATA: King Records (Starchild) (Owned by Kodansha (not public))


C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:  67%|██████▋   | 26/39 [00:27<00:14,  1.10s/it]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:  69%|██████▉   | 27/39 [00:28<00:12,  1.06s/it]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:  72%|███████▏  | 28/39 [00:29<00:10,  1.07it/s]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.t

NO DATA: NTT Docomo (Anime collaborations) (9437.T)


C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:  92%|█████████▏| 36/39 [00:36<00:02,  1.05it/s]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:  95%|█████████▍| 37/39 [00:37<00:01,  1.10it/s]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.to_period("Q")
Fetching data:  97%|█████████▋| 38/39 [00:38<00:00,  1.09it/s]C:\Users\Han Aga\AppData\Local\Temp\ipykernel_11100\522293094.py:83: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  hist["Quarter"] = hist.index.t


DONE
Files created:
- quarterly_linear_interpolated.csv
- quarterly_linear_interpolated.xml


In [ ]:
#My Anime List scarping script to get all animes by year including their mean and their number of

In [ ]:
import requests
import time
import pandas as pd
from datetime import datetime

all_anime = []
page = 1

print("Downloading entire MyAnimeList anime database...\n")

while True:
    url = f"https://api.jikan.moe/v4/anime?page={page}"
    print(f"Fetching page {page} ...")

    response = requests.get(url)

    # Rate limit handling
    if response.status_code == 429:
        print("Rate limited, waiting 2 seconds...")
        time.sleep(2)
        continue

    data = response.json()

    if "data" not in data or len(data["data"]) == 0:
        print("\n✔ Finished downloading all anime!")
        break

    for anime in data["data"]:
        title = anime.get("title")
        score = anime.get("score")
        genres = [g["name"] for g in anime.get("genres", [])]
        aired_from = anime.get("aired", {}).get("from")

        if aired_from:
            try:
                date = datetime.fromisoformat(aired_from.replace("Z", ""))
                year = date.year
                quarter = (date.month - 1) // 3 + 1
                year_quarter = f"{year}Q{quarter}"
            except Exception:
                year = None
                year_quarter = None
        else:
            year = None
            year_quarter = None

        all_anime.append({
            "title": title,
            "score": score,
            "year": year,
            "year_quarter": year_quarter
        })

    page += 1
    time.sleep(1.1)  # Jikan rate limit


# -----------------------
# DataFrames
# -----------------------

df = pd.DataFrame(all_anime)

from datetime import datetime

# -----------------------
# Determine last completed quarter
# -----------------------

now = datetime.now()
current_year = now.year
current_quarter = (now.month - 1) // 3 + 1

if current_quarter == 1:
    last_year = current_year - 1
    last_quarter = 4
else:
    last_year = current_year
    last_quarter = current_quarter - 1

last_completed_q = f"{last_year}Q{last_quarter}"

print(f"Using data up to: {last_completed_q}")

# -----------------------
# Filter dataframe
# -----------------------

df["sort_key"] = df["year_quarter"].str.replace("Q", "").astype(int)
max_key = int(f"{last_year}{last_quarter}")

df = df[df["sort_key"] <= max_key]
df = df.drop(columns="sort_key")

# Drop entries without quarter info
df = df.dropna(subset=["year_quarter"])

# Quarterly aggregation
quarterly_stats = df.groupby("year_quarter").agg(
    anime_count=("title", "count"),
    mean_score=("score", "mean")
).reset_index()

# Sort properly (YYYYQn)
quarterly_stats["sort_key"] = quarterly_stats["year_quarter"].str.replace("Q", "").astype(int)
quarterly_stats = quarterly_stats.sort_values("sort_key").drop(columns="sort_key")

# -----------------------
# Save files
# -----------------------

df.to_csv("all_anime_raw_quarterly.csv", index=False)
quarterly_stats.to_csv("anime_quarter_stats.csv", index=False)

print("\n✔ Saved:")
print(" - all_anime_raw_quarterly.csv")
print(" - anime_quarter_stats.csv")
print(quarterly_stats.head(10))



Fetching page 1 ...


KeyboardInterrupt: 

In [18]:

!pip install scipy

In [25]:
import pandas as pd
import numpy as np
import math

# -----------------------------
# 1. Load datasets
# -----------------------------
anime = pd.read_csv("anime_year_stats.csv")       # columns: year, anime_count, mean_score
market = pd.read_csv("yearly_values.csv")        # columns: Year, TotalValue_USD, etc.

# Ensure column names match
anime.rename(columns={"year": "Year"}, inplace=True)

# Merge on Year
df = anime.merge(market, on="Year", how="inner")

# -----------------------------
# 2. Compute Pearson correlation manually
# -----------------------------
def pearson_corr(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sqrt(np.sum((x - x_mean)**2) * np.sum((y - y_mean)**2))
    r = numerator / denominator
    return r

# -----------------------------
# 3. Compute two-tailed p-value from r
# -----------------------------
def pearson_p_value(r, n):
    if n < 3:
        return float('nan')
    t_stat = r * math.sqrt((n - 2) / (1 - r**2))
    # two-tailed p-value using normal approximation
    p = 2 * (1 - 0.5 * (1 + math.erf(abs(t_stat) / math.sqrt(2))))
    return p

# -----------------------------
# 4. Compute correlations and p-values
# -----------------------------
n = len(df)

# Mean score vs market value
r_score = pearson_corr(df["mean_score"], df["TotalValue_USD"])
p_score = pearson_p_value(r_score, n)

# Anime count vs market value
r_count = pearson_corr(df["anime_count"], df["TotalValue_USD"])
p_count = pearson_p_value(r_count, n)

# -----------------------------
# 5. Print results
# -----------------------------
print("=== CORRELATION RESULTS ===")
print(f"Mean Score ↔ Market Value:  r = {r_score:.4f}, p = {p_score:.4e}")
print(f"Anime Count ↔ Market Value: r = {r_count:.4f}, p = {p_count:.4e}")

# -----------------------------
# 6. Save results to file (UTF-8)
# -----------------------------
with open("correlation_results.txt", "w", encoding="utf-8") as f:
    f.write("=== CORRELATION RESULTS ===\n\n")
    f.write(f"Mean Score ↔ Market Value\n  r = {r_score:.4f}\n  p = {p_score:.4e}\n\n")
    f.write(f"Anime Count ↔ Market Value\n  r = {r_count:.4f}\n  p = {p_count:.4e}\n")

print("\nSaved results to correlation_results.txt")



=== CORRELATION RESULTS ===
Mean Score ↔ Market Value:  r = 0.3356, p = 5.0327e-03
Anime Count ↔ Market Value: r = 0.7723, p = 0.0000e+00

Saved results to correlation_results.txt


now the ml part for predictions

In [17]:
import pandas as pd
import numpy as np
from datetime import datetime
# -----------------------
# LOAD DATA
# -----------------------

anime = pd.read_csv("anime_quarter_stats.csv")
stocks = pd.read_csv("quarterly_linear_interpolated.csv")

anime = anime.set_index("year_quarter")
stocks = stocks.set_index("index")

industry = stocks["TotalValue"].dropna()

# align indices
quarters = anime.index.intersection(industry.index)
anime = anime.loc[quarters]
industry = industry.loc[quarters]

def next_quarter_label(last_q):
    year = int(last_q[:4])
    quarter = int(last_q[-1])

    now = datetime.now()
    current_q = (now.month - 1) // 3 + 1

    # if last known data is already behind current quarter,
    # predict the current quarter instead of skipping ahead
    if quarter < current_q or year < now.year:
        return f"{now.year}Q{current_q}"

    # otherwise predict the next quarter
    if quarter == 4:
        return f"{year + 1}Q1"
    else:
        return f"{year}Q{quarter + 1}"
# -----------------------
# HELPER: LINEAR REGRESSION (manual)
# -----------------------

def linear_regression(X, y, alpha=1.0):
    X = np.column_stack([np.ones(len(X)), X])
    I = np.eye(X.shape[1])
    I[0, 0] = 0
    return np.linalg.inv(X.T @ X + alpha * I) @ X.T @ y

# -----------------------
# 1️⃣ PREDICT ANIME COUNT (trend-based)
# -----------------------

t = np.arange(len(anime))
w_count = linear_regression(t.reshape(-1, 1), anime["anime_count"].values)
pred_anime_count = max(1, int(w_count @ np.array([1, len(t)])))

# -----------------------
# 2️⃣ PREDICT MEAN SCORE (trend-based)
# -----------------------

w_score = linear_regression(t.reshape(-1, 1), anime["mean_score"].values)
pred_mean_score = float(w_score @ np.array([1, len(t)]))

# -----------------------
# 3️⃣ INDUSTRY IMPACT MODEL
# -----------------------

rows = []

for i in range(1, len(quarters)):
    q = quarters[i]
    q_prev = quarters[i - 1]

    rows.append({
        "anime_count": anime.loc[q, "anime_count"],
        "mean_score": anime.loc[q, "mean_score"],
        "target": np.log(industry.loc[q] / industry.loc[q_prev])
    })

df = pd.DataFrame(rows).dropna()

X = df[["anime_count", "mean_score"]].values
y = df["target"].values

w_market = linear_regression(X, y)

# -----------------------
# USER OPTION
# -----------------------

print("\nChoose prediction mode:")
print("1 → Give anime count → predict money")
print("2 → Give money → predict anime count")
print("3 → Auto predict everything")

choice = input("Your choice (1/2/3): ").strip()

last_value = industry.iloc[-1]

if choice == "1":
    user_anime = float(input("Enter expected anime count: "))
    user_score = float(input("Enter expected mean score: "))

    X_pred = np.array([1, user_anime, user_score])
    delta = X_pred @ w_market
    predicted_value = last_value * np.exp(delta)

    print(f"\n📈 Predicted industry value: ${predicted_value:,.0f}")

elif choice == "2":
    anime_est = -1

    while anime_est < 0:
        try:
            user_money = float(input("Enter target industry value (USD): "))

            growth = np.log(user_money / last_value)

            # solve for anime count (mean score fixed)
            anime_est = (
                growth
                - w_market[0]
                - w_market[2] * pred_mean_score
            ) / w_market[1]

            if anime_est < 0:
                print("❌ Resulting anime count is negative — increase target value.")

        except ValueError:
            print("❌ Please enter a valid number.")

    print(f"\n🎬 Required anime count: {anime_est:.0f}")
    print(f"\n🎬 Estimated anime count needed: {anime_est:.1f}")
    print(f"Using mean score ≈ {pred_mean_score:.2f}")

else:
    X_pred = np.array([1, pred_anime_count, pred_mean_score])
    delta = X_pred @ w_market
    predicted_value = last_value * np.exp(delta)
    pct_change = (predicted_value / last_value - 1) * 100

    predicted_q = next_quarter_label(industry.index[-1])

    print(f"\n📊 AUTO {predicted_q} PREDICTION")
    print(f"Predicted anime count : {pred_anime_count}")
    print(f"Predicted mean score  : {pred_mean_score:.2f}")
    print(f"Predicted market val  : ${predicted_value:,.0f}")
    print(f"Expected change       : {pct_change:.2f}%")



Choose prediction mode:
1 → Give anime count → predict money
2 → Give money → predict anime count
3 → Auto predict everything

📊 AUTO 2026Q1 PREDICTION
Predicted anime count : 279
Predicted mean score  : 6.69
Predicted market val  : $49,129,349,272,829
Expected change       : 3.13%


In [ ]:
import os
import pandas as pd

# create folder if it doesn't exist
output_dir = "html_code"
os.makedirs(output_dir, exist_ok=True)

viz = pd.DataFrame({
    "quarter": industry.index,
    "industry_value": industry.values
})

viz.to_csv(f"{output_dir}/industry_timeseries.csv", index=False)

params = {
    "w0": w_market[0],
    "w1": w_market[1],
    "w2": w_market[2],
    "last_value": float(last_value),
    "pred_mean_score": float(pred_mean_score)
}

pd.Series(params).to_json(f"{output_dir}/model_params.json")

print("✔ Visualization files exported to /html_code")



✔ Visualization files exported


In [21]:
%cd "C:/Users/Han Aga/Desktop/dsa/DSA 210 anime/html code"
print("http://localhost:8000/")
!python -m http.server 8000

C:\Users\Han Aga\Desktop\dsa\DSA 210 anime\html code
http://localhost:8000/
^C
